# 02 - Exploratory Data Analysis

Explore and validate scraped BaT data before modeling.

**Inputs:**
- `data/raw/bat_listings.parquet`

**Outputs:**
- `data/processed/cleaned_listings.parquet`

In [ ]:
import logging
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from price_analysis.constants import GT_TRIMS, TRIM_MAPPING, WATER_COOLED_GENS
from price_analysis.data import clean_listings
from price_analysis.data.cleaning import get_summary_stats, prepare_model_data
from price_analysis.visualization.eda_viz import (
    plot_faceted_scatter,
    plot_lowess_curves,
    plot_price_by_category,
    plot_price_heatmap,
    plot_price_scatter,
)

logging.basicConfig(level=logging.INFO)
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
# Paths
DATA_DIR = Path("../data")
RAW_PATH = DATA_DIR / "raw" / "bat_listings.parquet"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH = PROCESSED_DIR / "cleaned_listings.parquet"

## Load Raw Data

In [ ]:
df_raw = pd.read_parquet(RAW_PATH)
print(f"Raw data: {len(df_raw)} listings")
display(df_raw.head())

In [ ]:
df_raw.info()

## Clean and Validate

In [ ]:
df = clean_listings(df_raw, drop_invalid=False)
display(df.head())

### New Features from Parsing

`clean_listings()` now re-parses `title_raw` to extract:
- **GTS trims**: Carrera GTS, Carrera 4 GTS, Targa 4 GTS (previously mislabeled as base Carrera/Targa)
- **Body style**: coupe, cabriolet, targa, speedster (defaulting to coupe for GT cars that omit it)

In [ ]:
stats = get_summary_stats(df)
print(f"Total listings: {stats['n_listings']}")
print(f"Valid listings: {stats['n_valid']}")
print(f"Date range: {stats['date_range'][0]} to {stats['date_range'][1]}")
print(f"Price range: ${stats['price_range'][0]:,} to ${stats['price_range'][1]:,}")
print(f"Median price: ${stats['price_median']:,}")
print(f"Median mileage: {stats['mileage_median']:,}")

## Missing Data Report

In [ ]:
missing = df.isna().sum()
missing_pct = (missing / len(df) * 100).round(1)
missing_report = pd.DataFrame({"missing": missing, "pct": missing_pct})
display(missing_report[missing_report["missing"] > 0].sort_values("missing", ascending=False))

## Distribution Plots

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Price distribution
sns.histplot(df["sale_price"].dropna() / 1000, bins=50, ax=axes[0, 0])
axes[0, 0].set_xlabel("Sale Price ($k)")
axes[0, 0].set_title("Price Distribution")

# Log price distribution
sns.histplot(df["log_price"].dropna(), bins=50, ax=axes[0, 1])
axes[0, 1].set_xlabel("Log(Sale Price)")
axes[0, 1].set_title("Log Price Distribution (for modeling)")

# Mileage distribution
sns.histplot(df["mileage"].dropna() / 1000, bins=50, ax=axes[1, 0])
axes[1, 0].set_xlabel("Mileage (k miles)")
axes[1, 0].set_title("Mileage Distribution")

# Age distribution
sns.histplot(df["age"].dropna(), bins=30, ax=axes[1, 1])
axes[1, 1].set_xlabel("Age (years)")
axes[1, 1].set_title("Age Distribution")

plt.tight_layout()
plt.show()

## Categorical Counts

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Generation counts
gen_counts = df["generation"].value_counts()
sns.barplot(x=gen_counts.values, y=gen_counts.index, ax=axes[0, 0], orient="h")
axes[0, 0].set_title("Listings by Generation")
axes[0, 0].set_xlabel("Count")

# Trim counts (top 12 to show GTS variants)
trim_counts = df["trim"].value_counts().head(12)
sns.barplot(x=trim_counts.values, y=trim_counts.index, ax=axes[0, 1], orient="h")
axes[0, 1].set_title("Listings by Trim (Top 12)")
axes[0, 1].set_xlabel("Count")

# Transmission counts
trans_counts = df["transmission"].value_counts()
sns.barplot(x=trans_counts.values, y=trans_counts.index, ax=axes[1, 0], orient="h")
axes[1, 0].set_title("Listings by Transmission")
axes[1, 0].set_xlabel("Count")

# Body style counts
body_counts = df["body_style"].value_counts()
sns.barplot(x=body_counts.values, y=body_counts.index, ax=axes[1, 1], orient="h", palette="Set2")
axes[1, 1].set_title("Listings by Body Style")
axes[1, 1].set_xlabel("Count")

plt.tight_layout()
plt.show()

## Price by Model Year & Sale Date

In [ ]:
# Focus on key generations
key_gens = ["991.1", "991.2", "992.1", "997.1", "997.2", "996.1", "996.2"]
df_key = df[df["generation"].isin(key_gens)].copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Price by model year (scatter)
ax = axes[0]
for gen in key_gens:
    subset = df_key[df_key["generation"] == gen]
    if len(subset) > 0:
        ax.scatter(
            subset["model_year"],
            subset["sale_price"] / 1000,
            label=gen,
            alpha=0.7,
            s=50,
        )
ax.set_xlabel("Model Year")
ax.set_ylabel("Sale Price ($k)")
ax.set_title("Price by Model Year")
ax.legend()

# Price over sale time (if enough data)
ax = axes[1]
has_data = False
for gen in key_gens:
    subset = df_key[df_key["generation"] == gen]
    if len(subset) >= 2:  # Need at least 2 points to show trend
        ax.scatter(
            subset["sale_date"],
            subset["sale_price"] / 1000,
            label=gen,
            alpha=0.7,
            s=50,
        )
        has_data = True

if has_data:
    ax.set_xlabel("Sale Date")
    ax.set_ylabel("Sale Price ($k)")
    ax.set_title("Price by Sale Date")
    ax.legend()
else:
    ax.text(
        0.5,
        0.5,
        "Not enough data for time series",
        ha="center",
        va="center",
        transform=ax.transAxes,
    )
    ax.set_title("Price by Sale Date (insufficient data)")

plt.tight_layout()
plt.show()

## Price by Mileage

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
for gen in key_gens:
    subset = df_key[df_key["generation"] == gen]
    ax.scatter(
        subset["mileage"] / 1000,
        subset["sale_price"] / 1000,
        label=gen,
        alpha=0.5,
        s=20,
    )

ax.set_xlabel("Mileage (k miles)")
ax.set_ylabel("Sale Price ($k)")
ax.set_title("Price vs Mileage by Generation")
ax.legend()
plt.tight_layout()
plt.show()

## Manual vs PDK Premium

In [ ]:
# Compare manual vs PDK prices
# With small datasets, compare overall (not controlling for generation+trim)
trans_df = df[df["transmission"].isin(["Manual", "PDK"])].copy()

if len(trans_df) > 0:
    # Overall comparison
    print("=== Overall Manual vs PDK ===")
    overall = trans_df.groupby("transmission")["sale_price"].agg(["count", "median", "mean"])
    overall["median"] = overall["median"].apply(lambda x: f"${x:,.0f}")
    overall["mean"] = overall["mean"].apply(lambda x: f"${x:,.0f}")
    display(overall)

    # By generation (if enough data)
    print("\n=== By Generation (where both exist) ===")
    gen_trans = trans_df.groupby(["generation", "transmission"])["sale_price"].agg(
        ["count", "median"]
    )
    gen_trans_wide = gen_trans.unstack("transmission")
    # Only show where we have both Manual and PDK
    gen_trans_wide = gen_trans_wide.dropna()
    if len(gen_trans_wide) > 0:
        display(gen_trans_wide)
    else:
        print("Not enough data - need same generation with both Manual and PDK listings")
else:
    print("No Manual or PDK listings in dataset")

## Correlation Matrix

In [ ]:
numeric_cols = ["sale_price", "log_price", "mileage", "age", "model_year", "sale_year"]
corr = df[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="viridis", center=0, ax=ax)
ax.set_title("Correlation Matrix")
plt.tight_layout()
plt.show()

## Filter & Group Data for Analysis

Focus on water-cooled 911s (996+) with simplified trim groupings:
- **Exclude GTs** (GT3, GT3 RS, GT2 RS, GT3 Touring) - analyzed separately
- **Combine Targas** with corresponding AWD coupes
- **Combine Turbo + Turbo S** into single "Turbo" tier

In [ ]:
# Create main analysis dataset (non-GT water-cooled)
df_main = df[
    df["generation"].isin(WATER_COOLED_GENS) & ~df["trim"].isin(GT_TRIMS) & df["trim"].notna()
].copy()

# Apply trim groupings
df_main["trim_grouped"] = df_main["trim"].replace(TRIM_MAPPING)

# Create GT dataset for separate analysis
df_gt = df[df["generation"].isin(WATER_COOLED_GENS) & df["trim"].isin(GT_TRIMS)].copy()

print(f"Main analysis: {len(df_main)} listings (non-GT water-cooled)")
print(f"GT models: {len(df_gt)} listings")
print(f"\nTrim distribution (after grouping):")
display(df_main["trim_grouped"].value_counts().to_frame("count"))

## Price Relationships by Key Variables

Visual breakdowns of how price varies with age, mileage, trim, transmission, and generation.

In [ ]:
# Price by Age - scatter with trend and box plots (water-cooled non-GT)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter with regression line
ax = axes[0]
df_age = df_main[df_main["age"].notna()].copy()
sns.regplot(
    data=df_age,
    x="age",
    y=df_age["sale_price"] / 1000,
    scatter_kws={"alpha": 0.3, "s": 20},
    line_kws={"color": "red"},
    ax=ax,
)
ax.set_xlabel("Age (years)")
ax.set_ylabel("Sale Price ($k)")
ax.set_title("Price vs Age (water-cooled non-GT)")

# Box plot by age buckets
ax = axes[1]
df_age["age_bucket"] = pd.cut(
    df_age["age"],
    bins=[0, 5, 10, 15, 20, 25, 30],
    labels=["0-5", "5-10", "10-15", "15-20", "20-25", "25-30"],
)
order = ["0-5", "5-10", "10-15", "15-20", "20-25", "25-30"]
sns.boxplot(
    data=df_age,
    x="age_bucket",
    y=df_age["sale_price"] / 1000,
    order=order,
    ax=ax,
)
ax.set_xlabel("Age (years)")
ax.set_ylabel("Sale Price ($k)")
ax.set_title("Price Distribution by Age Bucket")

plt.tight_layout()
plt.show()

In [ ]:
fig = plot_price_scatter(
    df_main,
    x_col="mileage",
    hue_col="generation",
    hue_order=WATER_COOLED_GENS,
    title="Price vs Mileage by Generation (water-cooled non-GT)",
)
plt.show()

In [ ]:
fig = plot_price_by_category(
    df_main,
    category_col="trim_grouped",
    title="Price Distribution by Trim (Targa->AWD, Turbo S->Turbo)",
)
plt.show()

In [ ]:
fig = plot_price_by_category(
    df_main,
    category_col="transmission",
    order=["Manual", "PDK", "Automatic", "Tiptronic"],
    palette="Set2",
    title="Price Distribution by Transmission (water-cooled non-GT)",
)
plt.show()

In [ ]:
fig = plot_price_by_category(
    df_main,
    category_col="body_style",
    order=["coupe", "cabriolet", "targa", "speedster"],
    palette="Set2",
    title="Price Distribution by Body Style (water-cooled non-GT)",
)
plt.show()

In [ ]:
fig = plot_price_by_category(
    df_main,
    category_col="generation",
    order=WATER_COOLED_GENS,
    palette="coolwarm",
    title="Price Distribution by Generation (water-cooled non-GT)",
)
plt.show()

In [ ]:
g = plot_faceted_scatter(
    df_main,
    x_col="age",
    facet_col="generation",
    col_order=WATER_COOLED_GENS,
    title="Price vs Age by Generation (water-cooled non-GT)",
)
plt.show()

In [ ]:
fig = plot_price_heatmap(
    df_main,
    row_col="trim_grouped",
    col_col="generation",
    col_order=WATER_COOLED_GENS,
    title="Median Price by Generation x Trim ($k) - water-cooled non-GT",
)
plt.show()

In [ ]:
fig = plot_lowess_curves(df_main, x_col="age", title="Price vs Age (LOWESS) - water-cooled non-GT")
plt.show()

In [ ]:
fig = plot_lowess_curves(
    df_main, x_col="mileage", title="Price vs Mileage (LOWESS) - water-cooled non-GT"
)
plt.show()

## GT Models (Separate Analysis)

GT3, GT3 RS, GT2 RS, GT3 Touring - track-focused variants with different pricing dynamics.

In [ ]:
# GT models summary
if len(df_gt) > 0:
    print(f"GT Models: {len(df_gt)} listings")
    print(f"\nBy trim:")
    display(df_gt["trim"].value_counts().to_frame("count"))

    print(f"\nBy generation:")
    display(
        df_gt.groupby(["generation", "trim"])["sale_price"].agg(["count", "median"]).sort_index()
    )
else:
    print("No GT models in dataset")

In [ ]:
# GT models visualization
if len(df_gt) >= 5:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Price by trim
    ax = axes[0]
    trim_order = df_gt.groupby("trim")["sale_price"].median().sort_values(ascending=False).index
    sns.boxplot(
        data=df_gt,
        x="trim",
        y=df_gt["sale_price"] / 1000,
        order=trim_order,
        palette="Reds",
        ax=ax,
    )
    sns.stripplot(
        data=df_gt,
        x="trim",
        y=df_gt["sale_price"] / 1000,
        order=trim_order,
        color="black",
        alpha=0.4,
        size=5,
        ax=ax,
    )
    ax.set_xlabel("GT Trim")
    ax.set_ylabel("Sale Price ($k)")
    ax.set_title("GT Models: Price by Trim")
    ax.tick_params(axis="x", rotation=45)

    # Price vs mileage for GTs
    ax = axes[1]
    df_gt_plot = df_gt[df_gt["mileage"].notna()].copy()
    df_gt_plot["mileage_k"] = df_gt_plot["mileage"] / 1000
    df_gt_plot["price_k"] = df_gt_plot["sale_price"] / 1000

    for trim in df_gt_plot["trim"].unique():
        subset = df_gt_plot[df_gt_plot["trim"] == trim]
        ax.scatter(subset["mileage_k"], subset["price_k"], label=trim, alpha=0.7, s=60)

    ax.set_xlabel("Mileage (k miles)")
    ax.set_ylabel("Sale Price ($k)")
    ax.set_title("GT Models: Price vs Mileage")
    ax.legend()

    plt.tight_layout()
    plt.show()
else:
    print("Not enough GT data for visualization")

## Prepare Model-Ready Data

In [ ]:
df_model = prepare_model_data(df)
print(f"Model-ready data: {len(df_model)} listings")
display(df_model.head())

In [ ]:
# Save cleaned data
df.to_parquet(OUTPUT_PATH, index=False)
print(f"Saved cleaned data to {OUTPUT_PATH}")